In [2]:
import pandas as pd

# Load the final combined data
file_path = 'combined_optimal_combinations_final.csv'
final_data = pd.read_csv(file_path)

# Filter for positive predicted returns
positive_returns_data = final_data[final_data['Predicted'] > 0]

# Initialize a dictionary to store individual portfolios
portfolios = {}

# Iterate over each sector in 'Left Sector' and create equal-weight portfolios
for sector, group in positive_returns_data.groupby('Left Sector'):
    # Add a new column 'Weight' with equal weights
    group['Weight'] = 1 / len(group)
    # Save the portfolio data for this sector
    portfolios[sector] = group

# Combine all sector portfolios into a single DataFrame
portfolio_data = pd.concat(portfolios.values(), ignore_index=True)

# Print the result for verification
print(portfolio_data)


   Left Sector Right Sector  Predicted  Actual    Weight
0           건설           보험     0.0003  0.0015  0.500000
1           건설          자동차     0.0005  0.0014  0.500000
2        경기소비재          반도체     0.0012  0.0004  0.200000
3        경기소비재           보험     0.0020  0.0015  0.200000
4        경기소비재           은행     0.0001  0.0039  0.200000
5        경기소비재          자동차     0.0007  0.0014  0.200000
6        경기소비재         헬스케어     0.0006  0.0004  0.200000
7         기계장비         정보기술     0.0001  0.0009  0.500000
8         기계장비         헬스케어     0.0015  0.0004  0.500000
9          반도체          자동차     0.0008  0.0014  0.333333
10         반도체         정보기술     0.0001  0.0009  0.333333
11         반도체         헬스케어     0.0004  0.0004  0.333333
12        방송통신          반도체     0.0008  0.0004  0.500000
13        방송통신           보험     0.0005  0.0015  0.500000
14          보험          반도체     0.0006  0.0004  0.166667
15          보험           보험     0.0005  0.0015  0.166667
16          보험           은행    

In [4]:
# Calculate portfolio returns based on Actual values and weights
portfolio_performance = []

for sector, group in portfolio_data.groupby('Left Sector'):
    # Calculate the weighted return for each portfolio
    weighted_return = (group['Weight'] * group['Actual']).sum()
    portfolio_performance.append({'Sector': sector, 'Portfolio Return': weighted_return})

# Convert to DataFrame
performance_df = pd.DataFrame(portfolio_performance)


performance_df


,Sector,Portfolio Return
0,건설,0.001450
1,경기소비재,0.001520
2,기계장비,0.000650
3,반도체,0.000900
4,방송통신,0.000950
5,보험,0.001417
6,에너지화학,-0.002600
7,은행,0.001450
8,자동차,0.000650
9,정보기술,0.000400


In [44]:
import pandas as pd

# 파일 경로
file_path = '섹터별종가.csv'

# 파일 읽기 (인코딩 문제 해결)
data = pd.read_csv(file_path, encoding='euc-kr')

# 불필요한 열 제거
data = data.drop(columns=[col for col in data.columns if "Unnamed" in col], errors='ignore')

# 날짜 열 이름 변경 및 날짜 형식으로 변환
data.rename(columns={data.columns[0]: "날짜"}, inplace=True)
data["날짜"] = pd.to_datetime(data["날짜"])

# 날짜별로 정렬 (수익률 계산 정확성을 위해)
data.sort_values(by="날짜", inplace=True)

# 모든 숫자 데이터를 float로 변환 (예상치 못한 데이터 형식 문제 해결)
data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# 수익률 계산 (전일 대비 변화율)
returns = data.iloc[:, 1:].pct_change()   # 퍼센트 변화율로 변환
returns["날짜"] = data["날짜"]  # 날짜 열 다시 추가

# 결측치가 있다면 0으로 채우기 (최종 확인용)
returns.fillna(0, inplace=True)

# 데이터 확인
print(returns.head(33))



        헬스케어      방송통신        보험     에너지화학        은행       자동차      정보기술  \
32  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
31 -0.004628  0.007913  0.015644 -0.007768  0.032202  0.005647 -0.000372   
30  0.011897  0.022856  0.041712  0.002143  0.041778  0.021447 -0.008003   
29 -0.010327 -0.010130 -0.021196 -0.007387 -0.027348 -0.013681 -0.015513   
28  0.014722 -0.000105 -0.008999  0.022043 -0.015458  0.003147  0.016813   
27  0.005699 -0.009592 -0.018686 -0.006867 -0.016862 -0.043937 -0.006719   
26 -0.006201  0.003937  0.011891 -0.006249  0.011836 -0.001550  0.007220   
25  0.026151  0.020136  0.034780  0.007906  0.032389  0.029989  0.034624   
24  0.002199 -0.001164 -0.006028  0.002042 -0.005908 -0.014073  0.000671   
23  0.031584 -0.013597 -0.006633  0.005120 -0.015101 -0.010538  0.000968   
22 -0.000215  0.010450 -0.017068 -0.005965 -0.003359  0.014435  0.015931   
21 -0.028636 -0.008700 -0.023458  0.005859 -0.038395  0.006276  0.014873   
20  0.001513

In [24]:
# Re-specify file paths
returns_data_path = '섹터별종가.csv'
construction_sector_path = '건설_sector.csv'

# Reload the files
returns_data = pd.read_csv(returns_data_path, encoding='euc-kr')
construction_sector_data = pd.read_csv(construction_sector_path, encoding='utf-8')

# Process the returns data
returns_data = returns_data.drop(columns=[col for col in returns_data.columns if "Unnamed" in col], errors='ignore')
returns_data.rename(columns={returns_data.columns[0]: "날짜"}, inplace=True)
returns_data["날짜"] = pd.to_datetime(returns_data["날짜"])
returns_data.sort_values(by="날짜", inplace=True)
returns_data.iloc[:, 1:] = returns_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
construction_returns = returns_data.iloc[:, 1:].pct_change() 
construction_returns["날짜"] = returns_data["날짜"]
construction_returns.fillna(0, inplace=True)

# Replace 4월 26일 data with predictions from construction_sector_data
date_to_replace = "2024-04-26"
for _, row in construction_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]   # Scale to percentage
    if sector in construction_returns.columns:
        construction_returns.loc[construction_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

# Set sectors not in construction_sector_data to 0 for 4월 26일
all_sectors = set(construction_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(construction_sector_data["Right Sector"])
for sector in sectors_to_zero:
    construction_returns.loc[construction_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

construction_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [25]:
# Re-specify file paths
returns_data_path = '섹터별종가.csv'
bank_sector_path = '은행_sector.csv'

# Reload the files
returns_data = pd.read_csv(returns_data_path, encoding='euc-kr')
bank_sector_data = pd.read_csv(bank_sector_path, encoding='utf-8')

# Process the returns data
returns_data = returns_data.drop(columns=[col for col in returns_data.columns if "Unnamed" in col], errors='ignore')
returns_data.rename(columns={returns_data.columns[0]: "날짜"}, inplace=True)
returns_data["날짜"] = pd.to_datetime(returns_data["날짜"])
returns_data.sort_values(by="날짜", inplace=True)
returns_data.iloc[:, 1:] = returns_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
bank_returns = returns_data.iloc[:, 1:].pct_change() 
bank_returns["날짜"] = returns_data["날짜"]
bank_returns.fillna(0, inplace=True)

# Replace 4월 26일 data with predictions from bank_sector_data
date_to_replace = "2024-04-26"
for _, row in bank_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]   # Scale to percentage
    if sector in bank_returns.columns:
        bank_returns.loc[bank_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

# Set sectors not in bank_sector_data to 0 for 4월 26일
all_sectors = set(bank_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(bank_sector_data["Right Sector"])
for sector in sectors_to_zero:
    bank_returns.loc[bank_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

bank_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [26]:
# Re-specify file paths
returns_data_path = '섹터별종가.csv'
consumer_goods_sector_path = '경기소비재_sector.csv'

# Reload the files
returns_data = pd.read_csv(returns_data_path, encoding='euc-kr')
consumer_goods_sector_data = pd.read_csv(consumer_goods_sector_path, encoding='utf-8')

# Process the returns data
returns_data = returns_data.drop(columns=[col for col in returns_data.columns if "Unnamed" in col], errors='ignore')
returns_data.rename(columns={returns_data.columns[0]: "날짜"}, inplace=True)
returns_data["날짜"] = pd.to_datetime(returns_data["날짜"])
returns_data.sort_values(by="날짜", inplace=True)
returns_data.iloc[:, 1:] = returns_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
consumer_goods_returns = returns_data.iloc[:, 1:].pct_change() 
consumer_goods_returns["날짜"] = returns_data["날짜"]
consumer_goods_returns.fillna(0, inplace=True)

# Replace 4월 26일 data with predictions from consumer_goods_sector_data
date_to_replace = "2024-04-26"
for _, row in consumer_goods_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]   # Scale to percentage
    if sector in consumer_goods_returns.columns:
        consumer_goods_returns.loc[consumer_goods_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

# Set sectors not in consumer_goods_sector_data to 0 for 4월 26일
all_sectors = set(consumer_goods_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(consumer_goods_sector_data["Right Sector"])
for sector in sectors_to_zero:
    consumer_goods_returns.loc[consumer_goods_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

consumer_goods_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [27]:
# Re-specify file paths
returns_data_path = '섹터별종가.csv'
healthcare_sector_path = '헬스케어_sector.csv'

# Reload the files
returns_data = pd.read_csv(returns_data_path, encoding='euc-kr')
healthcare_sector_data = pd.read_csv(healthcare_sector_path, encoding='utf-8')

# Process the returns data
returns_data = returns_data.drop(columns=[col for col in returns_data.columns if "Unnamed" in col], errors='ignore')
returns_data.rename(columns={returns_data.columns[0]: "날짜"}, inplace=True)
returns_data["날짜"] = pd.to_datetime(returns_data["날짜"])
returns_data.sort_values(by="날짜", inplace=True)
returns_data.iloc[:, 1:] = returns_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
healthcare_returns = returns_data.iloc[:, 1:].pct_change() 
healthcare_returns["날짜"] = returns_data["날짜"]
healthcare_returns.fillna(0, inplace=True)

# Replace 4월 26일 data with predictions from healthcare_sector_data
date_to_replace = "2024-04-26"
for _, row in healthcare_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]   # Scale to percentage
    if sector in healthcare_returns.columns:
        healthcare_returns.loc[healthcare_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

# Set sectors not in healthcare_sector_data to 0 for 4월 26일
all_sectors = set(healthcare_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(healthcare_sector_data["Right Sector"])
for sector in sectors_to_zero:
    healthcare_returns.loc[healthcare_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

healthcare_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [28]:
# 에너지화학
returns_data_path = '섹터별종가.csv'
energy_chemical_sector_path = '에너지화학_sector.csv'

# Reload files
returns_data = pd.read_csv(returns_data_path, encoding='euc-kr')
energy_chemical_sector_data = pd.read_csv(energy_chemical_sector_path, encoding='utf-8')

# Process returns data
returns_data = returns_data.drop(columns=[col for col in returns_data.columns if "Unnamed" in col], errors='ignore')
returns_data.rename(columns={returns_data.columns[0]: "날짜"}, inplace=True)
returns_data["날짜"] = pd.to_datetime(returns_data["날짜"])
returns_data.sort_values(by="날짜", inplace=True)
energy_chemical_returns = returns_data.iloc[:, 1:].pct_change()
energy_chemical_returns["날짜"] = returns_data["날짜"]
energy_chemical_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in energy_chemical_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in energy_chemical_returns.columns:
        energy_chemical_returns.loc[energy_chemical_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(energy_chemical_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(energy_chemical_sector_data["Right Sector"])
for sector in sectors_to_zero:
    energy_chemical_returns.loc[energy_chemical_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

energy_chemical_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [29]:
# 자동차
automobile_sector_path = '자동차_sector.csv'
automobile_sector_data = pd.read_csv(automobile_sector_path, encoding='utf-8')

# Process data
automobile_returns = returns_data.iloc[:, 1:].pct_change()
automobile_returns["날짜"] = returns_data["날짜"]
automobile_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in automobile_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in automobile_returns.columns:
        automobile_returns.loc[automobile_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(automobile_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(automobile_sector_data["Right Sector"])
for sector in sectors_to_zero:
    automobile_returns.loc[automobile_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

automobile_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [30]:
# 정보기술
info_tech_sector_path = '정보기술_sector.csv'
info_tech_sector_data = pd.read_csv(info_tech_sector_path, encoding='utf-8')

# Process data
info_tech_returns = returns_data.iloc[:, 1:].pct_change()
info_tech_returns["날짜"] = returns_data["날짜"]
info_tech_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in info_tech_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in info_tech_returns.columns:
        info_tech_returns.loc[info_tech_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(info_tech_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(info_tech_sector_data["Right Sector"])
for sector in sectors_to_zero:
    info_tech_returns.loc[info_tech_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

info_tech_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [31]:
# 기계장비
machinery_sector_path = '기계장비_sector.csv'
machinery_sector_data = pd.read_csv(machinery_sector_path, encoding='utf-8')

# Process data
machinery_returns = returns_data.iloc[:, 1:].pct_change()
machinery_returns["날짜"] = returns_data["날짜"]
machinery_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in machinery_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in machinery_returns.columns:
        machinery_returns.loc[machinery_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(machinery_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(machinery_sector_data["Right Sector"])
for sector in sectors_to_zero:
    machinery_returns.loc[machinery_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

machinery_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [32]:
# 반도체
semiconductor_sector_path = '반도체_sector.csv'
semiconductor_sector_data = pd.read_csv(semiconductor_sector_path, encoding='utf-8')

# Process data
semiconductor_returns = returns_data.iloc[:, 1:].pct_change()
semiconductor_returns["날짜"] = returns_data["날짜"]
semiconductor_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in semiconductor_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in semiconductor_returns.columns:
        semiconductor_returns.loc[semiconductor_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(semiconductor_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(semiconductor_sector_data["Right Sector"])
for sector in sectors_to_zero:
    semiconductor_returns.loc[semiconductor_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

semiconductor_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [33]:
# 방송통신
telecom_sector_path = '방송통신_sector.csv'
telecom_sector_data = pd.read_csv(telecom_sector_path, encoding='utf-8')

# Process data
telecom_returns = returns_data.iloc[:, 1:].pct_change()
telecom_returns["날짜"] = returns_data["날짜"]
telecom_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in telecom_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in telecom_returns.columns:
        telecom_returns.loc[telecom_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(telecom_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(telecom_sector_data["Right Sector"])
for sector in sectors_to_zero:
    telecom_returns.loc[telecom_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

telecom_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [34]:
# 보험
insurance_sector_path = '보험_sector.csv'
insurance_sector_data = pd.read_csv(insurance_sector_path, encoding='utf-8')

# Process data
insurance_returns = returns_data.iloc[:, 1:].pct_change()
insurance_returns["날짜"] = returns_data["날짜"]
insurance_returns.fillna(0, inplace=True)

# Replace and adjust returns
date_to_replace = "2024-04-26"
for _, row in insurance_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in insurance_returns.columns:
        insurance_returns.loc[insurance_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

all_sectors = set(insurance_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(insurance_sector_data["Right Sector"])
for sector in sectors_to_zero:
    insurance_returns.loc[insurance_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

insurance_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [36]:
# 증권
securities_sector_path = '증권_sector.csv'
securities_sector_data = pd.read_csv(securities_sector_path, encoding='utf-8')

# Process data
securities_returns = returns_data.iloc[:, 1:].pct_change()
securities_returns["날짜"] = returns_data["날짜"]
securities_returns.fillna(0, inplace=True)

# Replace 4월 26일 data with predictions from securities_sector_data
date_to_replace = "2024-04-26"
for _, row in securities_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in securities_returns.columns:
        securities_returns.loc[securities_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

# Set sectors not in securities_sector_data to 0 for 4월 26일
all_sectors = set(securities_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(securities_sector_data["Right Sector"])
for sector in sectors_to_zero:
    securities_returns.loc[securities_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

securities_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [37]:
# 철강
steel_sector_path = '철강_sector.csv'
steel_sector_data = pd.read_csv(steel_sector_path, encoding='utf-8')

# Process data
steel_returns = returns_data.iloc[:, 1:].pct_change()
steel_returns["날짜"] = returns_data["날짜"]
steel_returns.fillna(0, inplace=True)

# Replace 4월 26일 data with predictions from steel_sector_data
date_to_replace = "2024-04-26"
for _, row in steel_sector_data.iterrows():
    sector = row["Right Sector"]
    predicted_return = row["Predicted"]
    if sector in steel_returns.columns:
        steel_returns.loc[steel_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = predicted_return

# Set sectors not in steel_sector_data to 0 for 4월 26일
all_sectors = set(steel_returns.columns) - {"날짜"}
sectors_to_zero = all_sectors - set(steel_sector_data["Right Sector"])
for sector in sectors_to_zero:
    steel_returns.loc[steel_returns["날짜"] == pd.Timestamp(date_to_replace), sector] = 0

steel_returns.head(33)


,헬스케어,방송통신,보험,에너지화학,은행,자동차,정보기술,증권,철강,건설,경기소비재,기계장비,반도체,날짜
32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-12 00:00:00
31,-0.004628,0.007913,0.015644,-0.007768,0.032202,0.005647,-0.000372,0.038147,-0.011357,-0.010941,-0.003212,0.006513,-0.003891,2024-03-13 00:00:00
30,0.011897,0.022856,0.041712,0.002143,0.041778,0.021447,-0.008003,0.022037,0.001582,0.015168,0.008333,0.023444,-0.012967,2024-03-14 00:00:00
29,-0.010327,-0.010130,-0.021196,-0.007387,-0.027348,-0.013681,-0.015513,-0.024479,-0.007857,-0.000279,-0.002497,-0.019288,-0.006497,2024-03-15 00:00:00
28,0.014722,-0.000105,-0.008999,0.022043,-0.015458,0.003147,0.016813,-0.002087,0.011681,0.000544,-0.004179,0.025323,0.013431,2024-03-18 00:00:00
27,0.005699,-0.009592,-0.018686,-0.006867,-0.016862,-0.043937,-0.006719,-0.008431,-0.009423,-0.011644,-0.001949,-0.005543,-0.014548,2024-03-19 00:00:00
26,-0.006201,0.003937,0.011891,-0.006249,0.011836,-0.001550,0.007220,0.011833,-0.007122,-0.001339,0.004792,0.000319,0.013525,2024-03-20 00:00:00
25,0.026151,0.020136,0.034780,0.007906,0.032389,0.029989,0.034624,0.019872,0.012188,0.009071,0.009793,0.019809,0.034479,2024-03-21 00:00:00
24,0.002199,-0.001164,-0.006028,0.002042,-0.005908,-0.014073,0.000671,-0.003684,-0.004801,-0.007277,0.002262,0.000707,-0.008201,2024-03-22 00:00:00
23,0.031584,-0.013597,-0.006633,0.005120,-0.015101,-0.010538,0.000968,-0.021504,0.005143,-0.014959,-0.006780,-0.000380,-0.004496,2024-03-25 00:00:00


In [80]:
import numpy as np
import pandas as pd

# Example list of sector dataframes (replace with actual dataframes and their names)
sector_dataframes = {
    "Construction": construction_returns,
    "Bank": bank_returns,
    "ConsumerGoods": consumer_goods_returns,
    "Healthcare": healthcare_returns,
    "EnergyChemical": energy_chemical_returns,
    "Automobile": automobile_returns,
    "InfoTech": info_tech_returns,
    "Machinery": machinery_returns,
    "Semiconductor": semiconductor_returns,
    "Telecom": telecom_returns,
    "Insurance": insurance_returns,
    "Securities": securities_returns,
    "Steel": steel_returns,
}

# Combine all sectors into a single DataFrame for GMV optimization
# Using sector names as columns for identification
combined_returns = pd.concat(
    [df.set_index("날짜").iloc[:, :-1].add_prefix(f"{name}_") for name, df in sector_dataframes.items()],
    axis=1
)

# Replace 0 in 4월 26일 수익률 with 4월 25일 data
date_to_replace = pd.Timestamp("2024-04-26")
previous_date = pd.Timestamp("2024-04-25")

# Replace row-by-row at the DataFrame level
combined_returns.loc[date_to_replace] = combined_returns.loc[date_to_replace].mask(
    combined_returns.loc[date_to_replace] == 0, combined_returns.loc[previous_date]
)

# Select data from 3월 13일 to 4월 26일
start_date = pd.Timestamp("2024-03-13")
end_date = pd.Timestamp("2024-04-26")
data_for_gmv = combined_returns.loc[start_date:end_date]

# Drop columns with constant values (e.g., all 0 or single value)
data_for_gmv = data_for_gmv.loc[:, (data_for_gmv.std() != 0)]

# Calculate covariance matrix with regularization
cov_matrix = data_for_gmv.cov() + np.eye(len(data_for_gmv.columns)) * 1e-6  # Add small value for regularization

# GMV optimization
num_assets = len(cov_matrix)
ones = np.ones(num_assets)

# Solve for weights: w = Σ^(-1) * 1 / (1' * Σ^(-1) * 1)
inv_cov_matrix = np.linalg.inv(cov_matrix)
weights = np.dot(inv_cov_matrix, ones) / np.dot(ones.T, np.dot(inv_cov_matrix, ones))

# Ensure weights are between 0 and 1
weights = np.clip(weights, 0, 1)

# Normalize weights to sum to 1
weights /= weights.sum()

# Display results
portfolio_weights = pd.Series(weights, index=cov_matrix.columns, name="Weights")
portfolio_weights.head(15)






Construction_헬스케어     0.004298
Construction_방송통신     0.020266
Construction_보험       0.000000
Construction_에너지화학    0.000000
Construction_은행       0.000000
Construction_자동차      0.000000
Construction_정보기술     0.002635
Construction_증권       0.009149
Construction_철강       0.023950
Construction_건설       0.008717
Construction_경기소비재    0.007943
Construction_기계장비     0.000000
Bank_헬스케어             0.003893
Bank_방송통신             0.020161
Bank_보험               0.000000
Name: Weights, dtype: float64

In [81]:
import pandas as pd

# 포트폴리오 가중치를 데이터프레임 형태로 변환
portfolio_weights_df = portfolio_weights.reset_index()
portfolio_weights_df.columns = ['Column', 'Weight']

# 자산군 섹터 이름 추출
portfolio_weights_df['AssetSector'] = portfolio_weights_df['Column'].apply(lambda x: '_'.join(x.split('_')[1:]))
portfolio_weights_df['PortfolioType'] = portfolio_weights_df['Column'].apply(lambda x: x.split('_')[0])

# 포트폴리오 종류를 열로, 자산군 섹터 이름을 행으로 배치
weights_table = portfolio_weights_df.pivot(index='AssetSector', columns='PortfolioType', values='Weight')

# 결측치를 0으로 채움
weights_table = weights_table.fillna(0)

# 각 열(포트폴리오 종류)의 가중치 합을 구하고, 각 열을 해당 합으로 나누어 정규화
weights_table = weights_table.div(weights_table.sum(axis=0), axis=1)

# 결과 확인
weights_table.head(15)




PortfolioType,Automobile,Bank,Construction,ConsumerGoods,EnergyChemical,Healthcare,InfoTech,Insurance,Machinery,Securities,Semiconductor,Steel,Telecom
AssetSector,,,,,,,,,,,,,
건설,0.113994,0.113903,0.113267,0.112852,0.113669,0.112457,0.114072,0.113576,0.114537,0.114128,0.114071,0.112534,0.114003
경기소비재,0.103890,0.103556,0.103215,0.102475,0.104693,0.103577,0.103917,0.104490,0.103563,0.103406,0.103833,0.102358,0.102759
기계장비,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
방송통신,0.263026,0.263152,0.263339,0.260929,0.263985,0.259810,0.263370,0.263474,0.264442,0.261946,0.263748,0.261153,0.262920
보험,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
에너지화학,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
은행,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
자동차,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
정보기술,0.034042,0.034685,0.034237,0.045849,0.034530,0.045689,0.034652,0.033837,0.034003,0.033494,0.034065,0.045686,0.034099


In [82]:
import pandas as pd

# 영어 섹터 이름과 한국어 섹터 이름 매핑
sector_name_mapping = {
    "Construction": "건설",
    "Bank": "은행",
    "ConsumerGoods": "경기소비재",
    "Healthcare": "헬스케어",
    "EnergyChemical": "에너지화학",
    "Automobile": "자동차",
    "InfoTech": "정보기술",
    "Machinery": "기계장비",
    "Semiconductor": "반도체",
    "Telecom": "방송통신",
    "Insurance": "보험",
    "Securities": "증권",
    "Steel": "철강",
}

# 사진에서 가져온 섹터별 실제 수익률 데이터 (사용자 제공 데이터)
actual_returns = {
    "헬스케어": -0.002,
    "방송통신": 0.008353,
    "보험": 0.022823,
    "에너지화학": -0.00443,
    "은행": 0.05463358,
    "자동차": 0.00114,
    "정보기술": 0.016212,
    "증권": 0.020352,
    "철강": 0.003901,
    "건설": 0.007224,
    "경기소비재": 0.007339,
    "기계장비": 0.018176,
    "반도체": 0.019209
}

# 실제 수익률을 데이터프레임으로 변환
actual_returns_df = pd.DataFrame.from_dict(actual_returns, orient='index', columns=['ActualReturns'])

# 포트폴리오 가중치 테이블과 수익률 데이터 결합
weights_table.index.name = "AssetSector"
merged_table = weights_table.merge(actual_returns_df, left_index=True, right_index=True)

# 각 포트폴리오 성과 계산
portfolio_performance = (
    merged_table.drop(columns=["ActualReturns"])
    .multiply(merged_table["ActualReturns"], axis=0)
    .sum()
    .reset_index()
)

# 결과 열 이름 변경
portfolio_performance.columns = ["Sector", "Portfolio Return"]

# 영어 섹터 이름을 한국어로 변환
portfolio_performance["Sector"] = portfolio_performance["Sector"].map(sector_name_mapping)

# 결과 출력
portfolio_performance



,Sector,Portfolio Return
0,자동차,0.007923
1,은행,0.007933
2,건설,0.007852
3,경기소비재,0.008006
4,에너지화학,0.007920
5,헬스케어,0.008012
6,정보기술,0.007928
7,보험,0.007905
8,기계장비,0.007931
9,증권,0.007872


In [72]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Example list of sector dataframes (replace with actual dataframes and their names)
sector_dataframes = {
    "Construction": construction_returns,
    "Bank": bank_returns,
    "ConsumerGoods": consumer_goods_returns,
    "Healthcare": healthcare_returns,
    "EnergyChemical": energy_chemical_returns,
    "Automobile": automobile_returns,
    "InfoTech": info_tech_returns,
    "Machinery": machinery_returns,
    "Semiconductor": semiconductor_returns,
    "Telecom": telecom_returns,
    "Insurance": insurance_returns,
    "Securities": securities_returns,
    "Steel": steel_returns,
}

# Combine all sectors into a single DataFrame for optimization
combined_returns = pd.concat(
    [df.set_index("날짜").iloc[:, :-1].add_prefix(f"{name}_") for name, df in sector_dataframes.items()],
    axis=1
)

# Replace 0 in 4월 26일 수익률 with 4월 25일 data
date_to_replace = pd.Timestamp("2024-04-26")
previous_date = pd.Timestamp("2024-04-25")
combined_returns.loc[date_to_replace] = combined_returns.loc[date_to_replace].mask(
    combined_returns.loc[date_to_replace] == 0, combined_returns.loc[previous_date]
)

# Select data from 3월 13일 to 4월 26일
start_date = pd.Timestamp("2024-03-13")
end_date = pd.Timestamp("2024-04-26")
data_for_mvo = combined_returns.loc[start_date:end_date]

# Drop columns with constant values (e.g., all 0 or single value)
data_for_mvo = data_for_mvo.loc[:, (data_for_mvo.std() != 0)]

# Calculate covariance matrix
cov_matrix = data_for_mvo.cov() + np.eye(len(data_for_mvo.columns)) * 1e-6  # Add small value for regularization

# Calculate expected returns (mean returns) using 4월 26일 data as priority, fallback to 4월 25일
expected_returns = combined_returns.loc[date_to_replace]
expected_returns.fillna(combined_returns.loc[previous_date], inplace=True)

# Normalize expected returns to match selected columns
expected_returns = expected_returns[data_for_mvo.columns]

# Risk-free rate (for Sharpe Ratio calculation, assumed as 0.01 or 1%)
risk_free_rate = 0.01

# Define the Sharpe Ratio maximization objective function
def sharpe_ratio(weights):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (portfolio_return - risk_free_rate) / portfolio_volatility
    return -sharpe  # Negative because we are minimizing

# Constraints: weights sum to 1, bounds: 0 <= weights <= 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})  # Constraint that the sum of weights should be 1
bounds = [(0, 1) for _ in range(len(data_for_mvo.columns))]  # Bounds for each weight (0 <= weight <= 1)

# Initial guess for weights
initial_weights = np.array([1 / len(data_for_mvo.columns)] * len(data_for_mvo.columns))  # Evenly distributed initial weights

# Optimization
result = minimize(sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Optimized weights
optimized_weights = result.x

# Display results
portfolio_weights = pd.Series(optimized_weights, index=data_for_mvo.columns, name="Optimized Weights")
print(portfolio_weights)


Construction_헬스케어     2.142297e-14
Construction_방송통신     1.045778e-14
Construction_보험       0.000000e+00
Construction_에너지화학    0.000000e+00
Construction_은행       0.000000e+00
                          ...     
Steel_증권              0.000000e+00
Steel_철강              9.471590e-16
Steel_건설              6.418477e-16
Steel_경기소비재           7.716917e-15
Steel_기계장비            0.000000e+00
Name: Optimized Weights, Length: 156, dtype: float64


In [73]:
import pandas as pd

# 포트폴리오 가중치를 데이터프레임 형태로 변환
portfolio_weights_df = portfolio_weights.reset_index()
portfolio_weights_df.columns = ['Column', 'Weight']

# 자산군 섹터 이름 추출
portfolio_weights_df['AssetSector'] = portfolio_weights_df['Column'].apply(lambda x: '_'.join(x.split('_')[1:]))
portfolio_weights_df['PortfolioType'] = portfolio_weights_df['Column'].apply(lambda x: x.split('_')[0])

# 포트폴리오 종류를 열로, 자산군 섹터 이름을 행으로 배치
weights_table = portfolio_weights_df.pivot(index='AssetSector', columns='PortfolioType', values='Weight')

# 결측치를 0으로 채움
weights_table = weights_table.fillna(0)

# 각 열(포트폴리오 종류)의 가중치 합을 구하고, 각 열을 해당 합으로 나누어 정규화
weights_table = weights_table.div(weights_table.sum(axis=0), axis=1)

# 결과 확인
weights_table.head(15)



PortfolioType,Automobile,Bank,Construction,ConsumerGoods,EnergyChemical,Healthcare,InfoTech,Insurance,Machinery,Securities,Semiconductor,Steel,Telecom
AssetSector,,,,,,,,,,,,,
건설,0.000000e+00,0.070218,0.000000,0.057342,8.511941e-14,0.000000e+00,0.000000e+00,0.040396,0.192291,0.401058,3.301526e-14,3.851086e-15,0.000000
경기소비재,1.298857e-13,0.135221,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.198802,0.545517,0.047217,1.108488e-15,4.630150e-14,0.139940
기계장비,0.000000e+00,0.000000,0.281001,0.000000,1.360370e-14,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.679448
방송통신,4.273144e-14,0.367702,0.116287,0.000000,2.047737e-13,7.904094e-14,1.550843e-15,0.374007,0.000000,0.032957,6.095124e-14,2.717097e-14,0.021334
보험,1.000000e+00,0.231973,0.000000,0.012921,1.000000e+00,1.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,1.000000e+00,1.000000e+00,0.000000
에너지화학,1.009765e-13,0.194886,0.000000,0.175450,2.343958e-14,1.320870e-13,0.000000e+00,0.256926,0.000000,0.364843,6.242402e-14,3.737115e-14,0.063878
은행,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,2.440756e-15,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
자동차,0.000000e+00,0.000000,0.016223,0.000000,0.000000e+00,0.000000e+00,8.586881e-16,0.000000,0.012716,0.000000,0.000000e+00,0.000000e+00,0.000000
정보기술,1.229745e-14,0.000000,0.019454,0.734394,0.000000e+00,2.195483e-13,0.000000e+00,0.000000,0.249476,0.000000,0.000000e+00,0.000000e+00,0.007672


In [74]:
import pandas as pd

# 포트폴리오 가중치를 데이터프레임 형태로 변환
portfolio_weights_df = portfolio_weights.reset_index()
portfolio_weights_df.columns = ['Column', 'Weight']

# 자산군 섹터 이름 추출
portfolio_weights_df['AssetSector'] = portfolio_weights_df['Column'].apply(lambda x: '_'.join(x.split('_')[1:]))
portfolio_weights_df['PortfolioType'] = portfolio_weights_df['Column'].apply(lambda x: x.split('_')[0])

# 포트폴리오 종류를 열로, 자산군 섹터 이름을 행으로 배치
weights_table = portfolio_weights_df.pivot(index='AssetSector', columns='PortfolioType', values='Weight')

# 결측치를 0으로 채움
weights_table = weights_table.fillna(0)

# 각 열(포트폴리오 종류)의 가중치 합을 구하고, 각 열을 해당 합으로 나누어 정규화
weights_table = weights_table.div(weights_table.sum(axis=0), axis=1)

# 실제 수익률 (2024-04-26) 데이터 입력
actual_returns = {
    "헬스케어": -0.002,
    "방송통신": 0.008353,
    "보험": 0.022823,
    "에너지화학": -0.00443,
    "은행": 0.05463358,
    "자동차": 0.00114,
    "정보기술": 0.016212,
    "증권": 0.020352,
    "철강": 0.003901,
    "건설": 0.007224,
    "경기소비재": 0.007339,
    "기계장비": 0.018176,
    "반도체": 0.019209
}

# 실제 수익률을 데이터프레임으로 변환
actual_returns_df = pd.DataFrame.from_dict(actual_returns, orient='index', columns=['ActualReturns'])

# 가중치와 실제 수익률 결합
weights_table_with_returns = weights_table.merge(actual_returns_df, left_index=True, right_index=True)

# 각 포트폴리오별 성과 계산
portfolio_performance = weights_table_with_returns.drop(columns=["ActualReturns"]).multiply(weights_table_with_returns["ActualReturns"], axis=0).sum()

# 결과 출력
print(portfolio_performance)


Automobile        0.022823
Bank              0.009002
Construction      0.007219
ConsumerGoods     0.011875
EnergyChemical    0.022823
Healthcare        0.022823
InfoTech          0.022823
Insurance         0.004243
Machinery         0.009452
Securities        0.002503
Semiconductor     0.022823
Steel             0.022823
Telecom           0.013221
dtype: float64


In [78]:
import pandas as pd

# 실제 수익률 데이터 (이미지에서 제공된 실제 수익률)
actual_returns = {
    "헬스케어": -0.002,
    "방송통신": 0.008353,
    "보험": 0.022823,
    "에너지화학": -0.00443,
    "은행": 0.05463358,
    "자동차": 0.00114,
    "정보기술": 0.016212,
    "증권": 0.020352,
    "철강": 0.003901,
    "건설": 0.007224,
    "경기소비재": 0.007339,
    "기계장비": 0.018176,
    "반도체": 0.019209
}

# 실제 수익률을 데이터프레임으로 변환
actual_returns_df = pd.DataFrame.from_dict(actual_returns, orient='index', columns=['Portfolio Return'])

# 포트폴리오 이름을 한국어로 바꾸는 매핑
sector_name_mapping = {
    "자동차": "자동차",
    "은행": "은행",
    "건설": "건설",
    "경기소비재": "경기소비재",
    "에너지화학": "에너지화학",
    "헬스케어": "헬스케어",
    "정보기술": "정보기술",
    "보험": "보험",
    "기계장비": "기계장비",
    "증권": "증권",
    "반도체": "반도체",
    "방송통신": "방송통신",
    "철강": "철강"
}

# Sector 이름을 한국어로 변경
actual_returns_df['Sector'] = actual_returns_df.index

# 결과 출력: 영어 이름을 제외하고 한국어 이름만 출력
actual_returns_df['Sector'] = actual_returns_df['Sector'].map(sector_name_mapping)

# Display results without English names
actual_returns_df[[ 'Portfolio Return']]


,Portfolio Return
헬스케어,-0.002000
방송통신,0.008353
보험,0.022823
에너지화학,-0.004430
은행,0.054634
자동차,0.001140
정보기술,0.016212
증권,0.020352
철강,0.003901
건설,0.007224
